# <center> MobileNetV2 x1.4 Metrics and generation </center>

In this notebook adversarial examples are generated on each model version (quantized and fp) 

### Libraries and functions

In [ ]:
import numpy as np
import cv2
import os
import pickle
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torch.optim as optim
import time
from math import log10, sqrt
from torch.utils.data import DataLoader
from art.utils import load_cifar10
import random
import urllib
from torch.quantization import MovingAverageMinMaxObserver
from torch.ao.quantization.observer import MinMaxObserver
from torch.quantization import QuantStub, DeQuantStub
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10
from art.preprocessing.standardisation_mean_std import StandardisationMeanStdPyTorch
from art.attacks.evasion.hop_skip_jump import HopSkipJump
from scipy.stats.mstats import gmean
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)

def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    epsilon = 1e-10  # Small epsilon to avoid division by zero
    mse = max(mse, epsilon)  # Ensure MSE is not zero
    max_pixel = 1.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def test(model: nn.Module, dataloader: DataLoader, cuda=False) -> float:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            if cuda:
              inputs = inputs.cuda()
              labels = labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def normalize_np(img):
  img = torch.from_numpy(img)
  img = F.normalize(img, [0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
  return img.numpy()

def custom_collate(batch):
        # Combine a list of samples into a batch
        data, labels = zip(*batch)
        data = torch.stack(data)
        labels = torch.tensor(labels, dtype=torch.long)
        return data, labels

def evaluator(model, loader):
  model.eval()
  top_1 = 0
  top_5 = 0
  with torch.no_grad():
    for data in loader:
      inputs, labels = data
      outputs = model(inputs)

      _, predicted = torch.max(outputs, 1, keepdim=True)
      top_1 += torch.sum(predicted.view(-1) == labels).item()

      _, predicted_5 = torch.topk(outputs, k=5, dim=1)
      top_5 += torch.sum(predicted_5 == labels.unsqueeze(1)).item()

  return ("{:.2f}".format((top_1/400) * 100), "{:.2f}".format((top_5/400) * 100))

def evaluator_testset(model, loader):
  model.eval()
  top_1 = 0
  top_5 = 0
  with torch.no_grad():
    for data in loader:
      inputs, labels = data
      outputs = model(inputs)

      _, predicted = torch.max(outputs, 1, keepdim=True)
      top_1 += torch.sum(predicted.view(-1) == labels).item()

      _, predicted_5 = torch.topk(outputs, k=5, dim=1)
      top_5 += torch.sum(predicted_5 == labels.unsqueeze(1)).item()

  return ("{:.2f}".format((top_1/10000) * 100), "{:.2f}".format((top_5/10000) * 100))

### Dataset Preparation

In [ ]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

classes_cifar = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

y_test_set = np.zeros((400,),np.int8)

y_train_set = np.zeros((400,),np.int8)


for i in range(400):
        y_test_set[i] = np.where(y_test[i] == 1)[0][0]

for i in range(400):
        y_train_set[i] = np.where(y_train[i] == 1)[0][0]

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,shuffle=False, num_workers=2, pin_memory=True)

### MobileNetV2 Definition for quantized variants

In [ ]:
from functools import partial
from typing import Dict, Type, Any, Callable, Union, List, Optional
from torch import Tensor
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torch.ao.nn.quantized.modules.functional_modules import FloatFunctional

def _make_divisible(v: float, divisor: int, min_value: Optional[int] = None) -> int:
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        f_add = torch.nn.quantized.FloatFunctional()
        new_v = f_add.add(new_v, divisor)
        #new_v += divisor
    return new_v


class ConvBNActivation(nn.Sequential):
    def __init__(
        self,
        in_planes: int,
        out_planes: int,
        kernel_size: int = 3,
        stride: int = 1,
        groups: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        activation_layer: Optional[Callable[..., nn.Module]] = None,
        dilation: int = 1,
    ) -> None:
        padding = (kernel_size - 1) // 2 * dilation
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if activation_layer is None:
            activation_layer = nn.ReLU6
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, dilation=dilation, groups=groups,
                      bias=False),
            norm_layer(out_planes),
            activation_layer(inplace=True)
        )
        self.out_channels = out_planes


# necessary for backwards compatibility
ConvBNReLU = ConvBNActivation

class InvertedResidual(nn.Module):
    def __init__(
        self,
        inp: int,
        oup: int,
        stride: int,
        expand_ratio: int,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers: List[nn.Module] = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1, norm_layer=norm_layer))
        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim, norm_layer=norm_layer),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            norm_layer(oup),
        ])
        self.conv = nn.Sequential(*layers)
        self.out_channels = oup
        self._is_cn = stride > 1
        self.ff = torch.nn.quantized.FloatFunctional()

    def forward(self, x: Tensor) -> Tensor:
        if self.use_res_connect:
            #return x + self.conv(x)
            return self.ff.add(x, self.conv(x))
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(
        self,
        num_classes: int = 10,
        width_mult: float = 1.0,
        inverted_residual_setting: Optional[List[List[int]]] = None,
        round_nearest: int = 8,
        block: Optional[Callable[..., nn.Module]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        """
        Args:
            num_classes (int): Number of classes
            width_mult (float): Width multiplier - adjusts number of channels in each layer by this amount
            inverted_residual_setting: Network structure
            round_nearest (int): Round the number of channels in each layer to be a multiple of this number
            Set to 1 to turn off rounding
            block: Module specifying inverted residual building block for mobilenet
            norm_layer: Module specifying the normalization layer to use

        """
        super(MobileNetV2, self).__init__()

        if block is None:
            block = InvertedResidual

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        input_channel = 32
        last_channel = 1280

        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 16, 1, 1],
                [6, 24, 2, 1],  # NOTE: change stride 2 -> 1 for CIFAR10/100
                [6, 32, 3, 2],
                [6, 64, 4, 2],
                [6, 96, 3, 1],
                [6, 160, 3, 2],
                [6, 320, 1, 1],
            ]

        # only check the first element, assuming user knows t,c,n,s are required
        if len(inverted_residual_setting) == 0 or len(inverted_residual_setting[0]) != 4:
            raise ValueError("inverted_residual_setting should be non-empty "
                             "or a 4-element list, got {}".format(inverted_residual_setting))

        # building first layer
        input_channel = _make_divisible(input_channel * width_mult, round_nearest)
        self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)
        features: List[nn.Module] = [ConvBNReLU(3, input_channel, stride=1, norm_layer=norm_layer)]  # NOTE: change stride 2 -> 1 for CIFAR10/100
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c * width_mult, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t, norm_layer=norm_layer))
                input_channel = output_channel
        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1, norm_layer=norm_layer))
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # This exists since TorchScript doesn't support inheritance, so the superclass method
        # (this one) needs to have a name other than `forward` that can be accessed in a subclass
        x = self.features(x)
        # Cannot use "squeeze" as batch-size can be 1
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


### Floating Point and Quantized State Loading 

In [ ]:
adversarial_state1 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1umVJExm8VeVSqIpEWlCVhMbJrNZeU-aw&export=download&confirm=t&uuid=0")

adversarial_state2 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=12Ux4pWLxK4gTr54eMN-Mds4adPMvjfNX&export=download&confirm=t&uuid=0")

adversarial_state3 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=19Nxyh3nIwR0KdXv_ZBk8Ur-DtQzuuct3&export=download&confirm=t&uuid=0")




adversarial_quant_state = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1Z7raf4Eqpokh-SuzSdEIQRCXZSZOzhll&export=download&confirm=t&uuid=0")

adversarial_quant_state1 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=11kPJR9R95dfPx5RxrW_1B8TMKALTsgTO&export=download&confirm=t&uuid=0")

adversarial_quant_state2 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=18rWucu5ggvrwJ5IAsZl1WLSXsPET4WNN&export=download&confirm=t&uuid=0")

adversarial_quant_state3 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1Xc_-v33EZP0XLnUE4MIGCs-OeuCUcmik&export=download&confirm=t&uuid=0")

#### If you want your own files

In [ ]:
"""
If you want to manually load the files or load your own versions, use the code below updating your path accordingly

# Adversarial Model 1 https://drive.google.com/file/d/1umVJExm8VeVSqIpEWlCVhMbJrNZeU-aw/view?usp=sharing
# Adversarial Model 2 https://drive.google.com/file/d/12Ux4pWLxK4gTr54eMN-Mds4adPMvjfNX/view?usp=sharing
# Adversarial Model 3 https://drive.google.com/file/d/19Nxyh3nIwR0KdXv_ZBk8Ur-DtQzuuct3/view?usp=sharing


# Quantized Model https://drive.google.com/file/d/1Z7raf4Eqpokh-SuzSdEIQRCXZSZOzhll/view?usp=sharing
# Quantized Adversarial Model 1 https://drive.google.com/file/d/11kPJR9R95dfPx5RxrW_1B8TMKALTsgTO/view?usp=sharing
# Quantized Adversarial Model 2 https://drive.google.com/file/d/18rWucu5ggvrwJ5IAsZl1WLSXsPET4WNN/view?usp=sharing
# Quantized Adversarial Model 3 https://drive.google.com/file/d/1Xc_-v33EZP0XLnUE4MIGCs-OeuCUcmik/view?usp=sharing
"""


#path = "~/Downloads/"


#adversarial_state1 = os.path.join(path, "MobileNet_1it_CIFAR10_94.17acc.pkl")
#adversarial_state2 = os.path.join(path, "MobileNet_2it_CIFAR10_93.58acc.pkl")
#adversarial_state3 = os.path.join(path, "MobileNet_3it_CIFAR10_92.79acc.pkl")

#adversarial_quant_state1 = os.path.join(path, "MobileNet_Quantized_NoRT_94.03.pt")
#adversarial_quant_state1 = os.path.join(path, "MobileNet_Quantized_1RT_93.61.pt")
#adversarial_quant_state2 = os.path.join(path, "MobileNet_Quantized_2RT_93.28.pt")
#adversarial_quant_state3 = os.path.join(path, "MobileNet_Quantized_3RT_92.37.pt")

### Loading models to generate metrics

In [ ]:
mobilenet = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x1_4", pretrained=True)


mobilenet_1 = mobilenet
mobilenet_1 = mobilenet_1.load_state_dict(torch.load(adversarial_state1))


mobilenet_2 = mobilenet
mobilenet_2 = mobilenet_2.load_state_dict(torch.load(adversarial_state2))

mobilenet_3 = mobilenet
mobilenet_3 = mobilenet_3.load_state_dict(torch.load(adversarial_state3))



mobilenet_quant = MobileNetV2(width_mult = 1.4)
mobilenet_quant = mobilenet_quant.load_state_dict(torch.load(adversarial_quant_state,map_location=torch.device('cpu')))

mobilenet_quant_1 = MobileNetV2(width_mult = 1.4)
mobilenet_quant_1 = mobilenet_quant_1.load_state_dict(torch.load(adversarial_quant_state1,map_location=torch.device('cpu')))

mobilenet_quant_2 = MobileNetV2(width_mult = 1.4)
mobilenet_quant_2 = mobilenet_quant_2.load_state_dict(torch.load(adversarial_quant_state2,map_location=torch.device('cpu')))

mobilenet_quant_3 = MobileNetV2(width_mult = 1.4)
mobilenet_quant_3 = mobilenet_quant_3.load_state_dict(torch.load(adversarial_quant_state3,map_location=torch.device('cpu')))



### Metrics

#### Accuracy evaluation

In [ ]:
top1 , top5 = evaluator_testset(mobilenet, testloader)
print('Normal Model: Initial accuracy on test images: Top-1: {}%  Top-5: {}% - FP32'.format(top1, top5))

print("\n")

top1, top5 = evaluator_testset(mobilenet_quant, testloader)
print('Quantized: Accuracy on test images: Top-1: {}%  Top-5: {}% - INT8'.format(top1, top5))

print("\n")
print("\n")

top1 , top5 = evaluator_testset(mobilenet_1, testloader)
print('Adversarial Training 1 Model: Initial accuracy on test images: Top-1: {}%  Top-5: {}% - FP32'.format(top1, top5))

print("\n")

top1, top5 = evaluator_testset(mobilenet_quant_1, testloader)
print('Quantized Adversarial Training 1: Accuracy on test images: Top-1: {}%  Top-5: {}% - INT8'.format(top1, top5))

print("\n")
print("\n")

top1 , top5 = evaluator_testset(mobilenet_2, testloader)
print('Adversarial Training 2 Model: Initial accuracy on test images: Top-1: {}%  Top-5: {}% - FP32'.format(top1, top5))

print("\n")

top1, top5 = evaluator_testset(mobilenet_quant_2, testloader)
print('Quantized Adversarial Training 2: Accuracy on test images: Top-1: {}%  Top-5: {}% - INT8'.format(top1, top5))

print("\n")
print("\n")

top1 , top5 = evaluator_testset(mobilenet_3, testloader)
print('Adversarial Training 3 Model: Initial accuracy on test images: Top-1: {}%  Top-5: {}% - FP32'.format(top1, top5))

print("\n")

top1, top5 = evaluator_testset(mobilenet_quant_3, testloader)
print('Quantized Adversarial Training 3: Accuracy on test images: Top-1: {}%  Top-5: {}% - INT8'.format(top1, top5))


#### Adversarial Example Generation Function

There is no reason to run this. Adversarial examples on the test set are available on Google Drive.

In [ ]:
def generator(optimizer, model, ver):
    criterion = nn.CrossEntropyLoss()
    # Create the ART - PyTorch classifier
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0.0, 1.0),
        loss=criterion,
        optimizer=optimizer,
        preprocessing=((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)), #Normalization step shouldn't happen at the data. It's inserted here.
        input_shape=(3, 32, 32),
        nb_classes=10,
        device_type = "cpu"
    )
    print("HopSkipJump Attack Initialization")

    attack = HopSkipJump(classifier,64,targeted = False,verbose = True)

    x_test_eval_mobil = attack.generate(normalize_np(x_test[0:400]),y_test[0:400])

    with open("mobilenet_evaluation_{}.pkl".format(ver), 'wb') as f:
        pickle.dump(x_test_eval_mobil, f)


In [ ]:
optimizer = optim.SGD(mobilenet.parameters(), lr=0.1, momentum=0.9, dampening=0,weight_decay=0.0005, nesterov=True)
generator(optimizer, mobilenet, "fp_0")


optimizer = optim.SGD(mobilenet_1.parameters(), lr=0.0001, momentum=0.9, dampening=0,weight_decay=0.0005, nesterov=True)
generator(optimizer, mobilenet_1, "fp_1")

optimizer = optim.Adam(mobilenet_2.parameters(), lr=0.0001)
generator(optimizer, mobilenet_2, "fp_2")

optimizer = optim.Adam(mobilenet_3.parameters(), lr=5e-7)
generator(optimizer, mobilenet_3, "fp_3")


optimizer = optim.SGD(mobilenet_quant.parameters(), lr=0.1, momentum=0.9, dampening=0,weight_decay=0.0005, nesterov=True)
generator(optimizer, mobilenet_quant, "int_0")


optimizer = optim.SGD(mobilenet_quant_1.parameters(), lr=0.0001, momentum=0.9, dampening=0,weight_decay=0.0005, nesterov=True)
generator(optimizer, mobilenet_quant_1, "int_1")

optimizer = optim.Adam(mobilenet_quant_2.parameters(), lr=0.0001)
generator(optimizer, mobilenet_quant_2, "int_2")

optimizer = optim.Adam(mobilenet_quant_3.parameters(), lr=5e-7)
generator(optimizer, mobilenet_quant_3, "int_3")


#### Loading pre-generated examples

In [ ]:

x_test_mob_0 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1s7HCO5M5wJ144M9-90QHUW7yDLuYWKYS&export=download&confirm=t&uuid=0")
x_test_mob_1 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1D2193zDfSOKg8biFTOoPnLQgSaigMh2e&export=download&confirm=t&uuid=0")
x_test_mob_2 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1zUyk_GAn2-H7qoqnm-N-VGY2Wb9hTXkl&export=download&confirm=t&uuid=0")
x_test_mob_3 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1y_4dgV6Sn5KnjqayI-VABvFPxT0bVOyS&export=download&confirm=t&uuid=0")

x_test_mob_q0 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1ltIXexqG4u7VCIYqXG7iH6BOHz649TIt&export=download&confirm=t&uuid=0")
x_test_mob_q1 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=10NIyPfZGfFBjumgZOIqsSE2-hJxrJ6ku&export=download&confirm=t&uuid=0")
x_test_mob_q2 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1--IwSY5aoeeukky2y7UdvwLAoOqJcT-W&export=download&confirm=t&uuid=0")
x_test_mob_q3 = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1bkIpoQ7mmLWb-A7Z78b0HVDwxwhXunC4&export=download&confirm=t&uuid=0")



"""
If you want to manually load the files or load your own versions, use the code below updating your path accordingly

# Initial   https://drive.google.com/file/d/1s7HCO5M5wJ144M9-90QHUW7yDLuYWKYS/view?usp=sharing
# Adversarial 1 https://drive.google.com/file/d/1D2193zDfSOKg8biFTOoPnLQgSaigMh2e/view?usp=sharing
# Adversarial 2 https://drive.google.com/file/d/1zUyk_GAn2-H7qoqnm-N-VGY2Wb9hTXkl/view?usp=sharing
# Adversarial 3 https://drive.google.com/file/d/1y_4dgV6Sn5KnjqayI-VABvFPxT0bVOyS/view?usp=sharing


# Quantized Initial https://drive.google.com/file/d/1ltIXexqG4u7VCIYqXG7iH6BOHz649TIt/view?usp=sharing
# Quantized Adversarial 1 https://drive.google.com/file/d/10NIyPfZGfFBjumgZOIqsSE2-hJxrJ6ku/view?usp=sharing
# Quantized Adversarial 2 https://drive.google.com/file/d/1--IwSY5aoeeukky2y7UdvwLAoOqJcT-W/view?usp=sharing
# Quantized Adversarial 3 https://drive.google.com/file/d/1bkIpoQ7mmLWb-A7Z78b0HVDwxwhXunC4/view?usp=sharing
"""


#path = "~/Downloads/"


#x_test_mob_0 = os.path.join(path, "x_test_hop_mob_fp_400.pkl")
#x_test_mob_1 = os.path.join(path, "MobileNetV2_fp_CIFAR10_1_ex.pkl")
#x_test_mob_2 = os.path.join(path, "MobileNetV2_fp_CIFAR10_2_ex.pkl")
#x_test_mob_3 = os.path.join(path, "MobileNetV2_fp_CIFAR10_3_ex.pkl")

#x_test_mob_q0 = os.path.join(path, "x_test_hop_mobile_quant_400_psnr_check_No_retrain.pkl")
#x_test_mob_q1 = os.path.join(path, "x_test_hop_mobile_quant_400_psnr_check_1_retrain.pkl")
#x_test_mob_q2 = os.path.join(path, "x_test_hop_mobile_quant_400_psnr_check_2_retrain.pkl")
#x_test_mob_q3 = os.path.join(path, "x_test_hop_mobile_quant_400_psnr_check_3_retrain.pkl")

#### PSNR Data Evaluation

In [ ]:
mob_c_ts_0 = []
mob_c_ts_1 = []
mob_c_ts_2 = []
mob_c_ts_3 = []

mob_c_ts_q0 = []
mob_c_ts_q1 = []
mob_c_ts_q2 = []
mob_c_ts_q3 = []


for l in range(0,400):
    mob_c_ts_0.append(PSNR(x_test[l],x_test_mob_0[l]))
    mob_c_ts_1.append(PSNR(x_test[l],x_test_mob_1[l]))
    mob_c_ts_2.append(PSNR(x_test[l],x_test_mob_2[l]))
    mob_c_ts_3.append(PSNR(x_test[l],x_test_mob_3[l]))

    mob_c_ts_q0.append(PSNR(x_test[l],x_test_mob_q0[l]))
    mob_c_ts_q1.append(PSNR(x_test[l],x_test_mob_q1[l]))
    mob_c_ts_q2.append(PSNR(x_test[l],x_test_mob_q2[l]))
    mob_c_ts_q3.append(PSNR(x_test[l],x_test_mob_q3[l]))

In [ ]:
psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_0)))
print("MobileNet FloatPoint CIFAR-10 Test 0 PSNR: {}dB".format(psnr))
print("\n")
psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_1)))
print("MobileNet FloatPoint CIFAR-10 Test 1 PSNR: {}dB".format(psnr))
print("\n")
psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_2)))
print("MobileNet FloatPoint CIFAR-10 Test 2 PSNR: {}dB".format(psnr))
print("\n")
psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_3)))
print("MobileNet FloatPoint CIFAR-10 Test 3 PSNR: {}dB".format(psnr))
print("\n")



psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_q0)))
print("MobileNet Int8 CIFAR-10 Test 0 PSNR: {}dB".format(psnr))
print("\n")
psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_q1)))
print("MobileNet Int8 CIFAR-10 Test 1 PSNR: {}dB".format(psnr))
print("\n")
psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_q2)))
print("MobileNet Int8 CIFAR-10 Test 2 PSNR: {}dB".format(psnr))
print("\n")
psnr = gmean(list(filter(lambda x: x != 0, mob_c_ts_q3)))
print("MobileNet Int8 CIFAR-10 Test 3 PSNR: {}dB".format(psnr))
print("\n")